In [ ]:
import os
import json 
import base64 
import tensorflow as tf 
from tensorflow import keras 

In [11]:
# Encoding 1000 images to JSON 

encoding = 'utf-8'
output = 'project/dogsCats/output.json'
filenames = os.listdir("project/dogsCats/test1")
out = dict()
for filename in filenames: 
    with open('project/dogsCats/test1/'+filename, 'rb') as open_file: 
        byte_data = open_file.read()
        base64bytes = base64.b64encode(byte_data)
        base64string = base64bytes.decode(encoding)
        out[filename]= base64string

json.dump(out, open(output, 'w'))


11


In [19]:
# Decoding JSON 

inp = json.load(open(output, 'r')) 
name2Image = dict()

for filename, base64string in inp.items():
    image = base64.b64decode(base64string.encode('utf-8'))
    with open(f'project/dogsCats/test2/{filename}', "wb") as fh:
        fh.write(image)



In [ ]:
import numpy as np 
import pandas as pd 

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

import os

In [ ]:
filenames = os.listdir("dogsCats/train/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append('1')
    else:
        categories.append('0')

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})
df.head()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

image_size = 224
input_shape = (image_size, image_size, 3)

epochs = 5
batch_size = 16

pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")
    
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False

for layer in pre_trained_model.layers[15:]:
    layer.trainable = True
    
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
    
# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.1)
train_df = train_df.reset_index()
validate_df = validate_df.reset_index()

# validate_df = validate_df.sample(n=100).reset_index() # use for fast testing code purpose
# train_df = train_df.sample(n=1800).reset_index() # use for fast testing code purpose

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "dogsCats/train/train/", 
    x_col='filename',
    y_col='category',
    class_mode='binary',
    target_size=(image_size, image_size),
    batch_size=batch_size
)

In [ ]:
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size)

In [ ]:
loss, accuracy = model.evaluate_generator(validation_generator, total_validate//batch_size, workers=12)
print("Test: accuracy = %f  ;  loss = %f " % (accuracy, loss))

In [1]:
# Make a prediction with the Trained Model 
#trained_model = tf.keras.models.load_model('project/finalized_model.sav')
test_filenames = os.listdir("dogsCats/test2")

for filename in test_filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append('1')
    else:
        categories.append('0')


test_df = pd.DataFrame({
    'filename': test_filenames,
    'true_category': categories})
nb_samples = test_df.shape[0]

test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "dogsCats/test2/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    batch_size=batch_size,
    target_size=(image_size, image_size),
    shuffle=False
)

predict = model.predict_generator(name2image.values(), steps=np.ceil(nb_samples/batch_size))
threshold = 0.5
test_df['predicted_probability'] = (predict)
test_df['predicted_category'] = np.where(predict > threshold, 1,0)
test_df['correct_prediction'] = df.apply(lambda row: int(row.true_category == row.predicted_category), axis = 1)

In [ ]:
# Export data as JSON 

test_df['filename', 'predicted_probability'].to_json(r'final_Output.json')

In [ ]:
# Convert resulting data frame to Excel 

df_json=pd.read(test_df)
df_json.to_excel('OutputDogsCats.xlsx')